## A Basic Word Count Example With Spark

In [0]:
import pyspark.sql.functions as F
from pyspark.sql import Row

poor_reviews = spark.sparkContext.textFile(
  "s3://dso-560-nlp-text-analytics/poor_amazon_toy_reviews.txt")

### Word Count
The below code takes `poor_reviews`, iterates through each line, and splits each line (a string) by whitespace.
`flatMap` will flatten out the entire list of lists.

Underneath, a `flatMap`, `map`, and `reduce` operation looks like this ([original blog link](https://www.jowanza.com/blog/2017/8/11/partitions-in-apache-spark)):

![link](https://images.squarespace-cdn.com/content/v1/58a3db0903596e0fdbb27391/1502484724596-T4IEBESUF806VN5CZOEZ/image-asset.png?format=2500w)

In [0]:
"""
We first are telling each worker to split each line on white space (tokenizing, although remember, this is not the fully correct way to do it!)
each word is sent to a worker node that has the same keys (words) as it. 

Then the reduceByKey step performs the actual counts and sends the results back to the master node. We then sort for the top 10 results.
"""
poor_reviews.\
  flatMap(lambda line: line.split(" ")).\
  map(lambda word: (word, 1)).\
  reduceByKey(lambda a, b: a + b).\
  sortBy(lambda x: -x[1]).\
  take(10)

Out[8]: [('the', 18993),
 ('and', 10909),
 ('I', 9629),
 ('to', 9589),
 ('a', 9204),
 ('it', 8890),
 ('', 7347),
 ('of', 6692),
 ('not', 5461),
 ('is', 5449)]

#### Convert to DataFrame and Filter for Age Groups
It's often much easier to work with data within Spark as `DataFrames`, which are essentially similar in its API to a Pandas dataframe. However, a Spark `DataFrame` can be distributed across thousands of worker nodes.

In [0]:
# convert from RDDs to a Spark DataFrame
poor_reviews_df = poor_reviews.map(Row("review")).toDF()

# filter for all reviews that contain "... year old" in the phrase, show first 3
poor_reviews_df.filter(F.col("review").rlike(r'[0-9] (?:year|yr) old')).take(3)

Out[9]: [Row(review='Very disappointed. Expected a &#34;mega&#34; set only to receive this tiny set. Good for a 3 yr old.'),
 Row(review="My nearly 3 yr old granddaughter is not as fascinated as I thought she would be when I read the reviews when shopping for it.  It's turning out to be an unused toy."),
 Row(review='"This toy is HORRIBLE.  I bought it for my 4 yr old to play with and he got so frustrated when the legs fell off. We would no sooner get one leg on and another, or two, would fall off.  The ant is very cool and now that the legs, head and wings are super glued together.<br />I did get to glue it so that only the legs and abdomen touch the ground.  Such a waste for a really cool toy."')]

#### Replace Malformed HTML Encodings Using Regex

In [0]:
MALFORMED_CHARACTER_ENCODINGS_REGEX = r'\b&#\d+\b'

poor_reviews_df.withColumn("cleaned_text", 
                           F.regexp_replace(
                             F.col("review"), MALFORMED_CHARACTER_ENCODINGS_REGEX, "")).show(3)

+--------------------+--------------------+
|              review|        cleaned_text|
+--------------------+--------------------+
|Do not buy these!...|Do not buy these!...|
|Showed up not how...|Showed up not how...|
|You need expansio...|You need expansio...|
+--------------------+--------------------+
only showing top 3 rows



#### Convert to SQL View and Query Using SQL
Sometimes, it's easier to write our transformations in SQL.

In [0]:
poor_reviews_df.createOrReplaceTempView("poor_reviews")
# filter for reviews where the exclamation point repeats 4-7 times
spark.sql("""
SELECT *
FROM poor_reviews
WHERE review RLIKE '!{4,7}'
""").collect()

Out[11]: [Row(review="When we blew it up it was top heavy on one side. Therefore it would not throw or bounce right. It's like the thickness of the rubber is off. My kids never played with it. Total waste of money!!!!!!!!"),
 Row(review='"Please ignore original review. This backpack did not make it past the first week of school before the strap already tore from the bag and the only thing in it was a notebook and empty binder! What a waste!!!!  For all the more I paid for this backpack, this is such a great deal! Bought this for my daughter who is in the 7th grade for school. We just received it, so can\'t say much for durability, but it looks pretty sturdy. You cannot fit a lot of large/heavy books, but it will hold a decent amount. It\'s very cute and nicely made. She loves it."'),
 Row(review='I got a knock-off doll! Such a crappy product! I am livid!!!!!'),
 Row(review='"i ordered July 30 and received Sept 28.  The kids were SUPER excited!  The package came with 3 attachments, each

## Example 2: Simple Sentence Translation Example Using Distributed Clusters
In this use case, we will use transformers to convert English text into French. 
Taken from [Databricks documentation](https://docs.databricks.com/machine-learning/reference-solutions/natural-language-processing.html).

### Imports
We need to import several built in tools form `sparknlp`, as well as the `Pipeline` object from `pyspark.ml` ([docs](https://spark.apache.org/docs/latest/ml-pipeline.html)). A `Pipeline` object takes in `DataFrames` and then performs operations on the dataframe.

In [0]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SentenceDetectorDLModel, MarianTransformer
from pyspark.ml import Pipeline

## Set Up Our Pipeline
We'll string together three different transformations for our pipeline.

### DocumentAssembler ([link](https://nlp.johnsnowlabs.com/api/com/johnsnowlabs/nlp/DocumentAssembler))
> "Prepares data into a format that is processable by Spark NLP. This is the entry point for every Spark NLP pipeline. The DocumentAssembler can read either a String column or an Array[String]." 

### SentenceDetectorDLModel ([link](https://nlp.johnsnowlabs.com/api/com/johnsnowlabs/nlp/annotators/sentence_detector_dl/SentenceDetectorDLModel))
> "Annotator that detects sentence boundaries using a deep learning approach."

### MarianTransformer ([link](https://nlp.johnsnowlabs.com/docs/en/transformers))

> "Marian is an efficient, free Neural Machine Translation framework written in pure C++ with minimal dependencies. It is mainly being developed by the Microsoft Translator team. Many academic (most notably the University of Edinburgh and in the past the Adam Mickiewicz University in Poznań) and commercial contributors help with its development. MarianTransformer uses the models trained by MarianNMT. It is currently the engine behind the Microsoft Translator Neural Machine Translation services and being deployed by many companies, organizations and research projects."

In [0]:
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx") \
  .setInputCols("document").setOutputCol("sentence")

marian_transformer = MarianTransformer.pretrained() \
  .setInputCols("sentence").setOutputCol("translation")

pipeline = Pipeline().setStages([document_assembler,  sentence_detector, marian_transformer])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_en_fr download started this may take some time.
Approximate size to download 378.7 MB
[OK!]


Now, we'll go and generate some documents data.

In [0]:
data = spark.createDataFrame([["You can use Spark NLP to translate text. " + \
                               "This example pipeline translates English to French. I watched the World Cup today with my Friends"]]).toDF("text")

## Perform the Actual Transformation

In [0]:
# Create a pipeline model that can be reused across multiple data frames
model = pipeline.fit(data)

# You can use the model on any data frame that has a “text” column
result = model.transform(data)

display(result.select("text", "translation.result"))

text,result
You can use Spark NLP to translate text. This example pipeline translates English to French. I watched the World Cup today with my Friends,"List(Vous pouvez utiliser Spark NLP pour traduire du texte., Cet exemple de pipeline traduit l'anglais au français., J'ai regardé la Coupe du Monde aujourd'hui avec mes amis)"


### Download the Datasets

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.train
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.testa

# Example 3: Training A Named Entity Recogition Model Using Spark NLP Model Training and MlFlow 
In this example, we will train a model to identify named entities from text. 
From [Databricks examples](https://docs.databricks.com/machine-learning/reference-solutions/natural-language-processing.html#train-and-use-a-named-entity-recognition-model-using-spark-nlp-and-mlflow).

In [0]:
%sh
python -m spacy download en_core_web_sm

  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.0
    Uninstalling en-core-web-sm-2.2.0:
      Successfully uninstalled en-core-web-sm-2.2.0
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-15083cd9-4db1-4af6-97ff-45eb729d5bab/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

### Baseline Example of Named Entities

In [0]:
# opening text of an article from NYT: https://www.nytimes.com/2022/12/03/health/elderly-living-alone.html
text = "Lynne Ingersoll and her cat, Jesse, spent a quiet Thanksgiving Day together in her small bungalow in Blue Island, Ill. A retired librarian, Ms. Ingersoll never married or had children. At 77, she has outlived her parents, three partners, her two closest friends, five dogs and eight cats. When her sister died three years ago, Ms. Ingersoll joined the ranks of older Americans considered “kinless”: without partners or spouses, children or siblings. Covid-19 has largely suspended her occasional get-togethers with friends, too. Now, she said, “my social life consists of doctors and store clerks — that’s a joke, but it’s pretty much true.”"


doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Lynne Ingersoll 0 15 PERSON
Jesse 29 34 PERSON
Thanksgiving Day 50 66 EVENT
Blue Island 101 112 GPE
Ill. 114 118 GPE
Ingersoll 144 153 PERSON
77 188 190 DATE
three 222 227 CARDINAL
two 242 245 CARDINAL
five 263 267 CARDINAL
eight 277 282 CARDINAL
three years ago 310 325 DATE
Ingersoll 331 340 PERSON
Americans 367 376 NORP


### Load in the Test/Training Data

In [0]:
from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, 'file:/databricks/driver/eng.train')
test_data = CoNLL().readDataset(spark, 'file:/databricks/driver/eng.testa')

In [0]:
display(training_data)

text,document,sentence,token,pos,label
EU rejects German call to boycott British lamb .,"List(List(document, 0, 47, EU rejects German call to boycott British lamb ., Map(training -> true), List()))","List(List(document, 0, 47, EU rejects German call to boycott British lamb ., Map(sentence -> 0), List()))","List(List(token, 0, 1, EU, Map(sentence -> 0), List()), List(token, 3, 9, rejects, Map(sentence -> 0), List()), List(token, 11, 16, German, Map(sentence -> 0), List()), List(token, 18, 21, call, Map(sentence -> 0), List()), List(token, 23, 24, to, Map(sentence -> 0), List()), List(token, 26, 32, boycott, Map(sentence -> 0), List()), List(token, 34, 40, British, Map(sentence -> 0), List()), List(token, 42, 45, lamb, Map(sentence -> 0), List()), List(token, 47, 47, ., Map(sentence -> 0), List()))","List(List(pos, 0, 1, NNP, Map(word -> EU, sentence -> 0), List()), List(pos, 3, 9, VBZ, Map(word -> rejects, sentence -> 0), List()), List(pos, 11, 16, JJ, Map(word -> German, sentence -> 0), List()), List(pos, 18, 21, NN, Map(word -> call, sentence -> 0), List()), List(pos, 23, 24, TO, Map(word -> to, sentence -> 0), List()), List(pos, 26, 32, VB, Map(word -> boycott, sentence -> 0), List()), List(pos, 34, 40, JJ, Map(word -> British, sentence -> 0), List()), List(pos, 42, 45, NN, Map(word -> lamb, sentence -> 0), List()), List(pos, 47, 47, ., Map(word -> ., sentence -> 0), List()))","List(List(named_entity, 0, 1, B-ORG, Map(word -> EU, sentence -> 0), List()), List(named_entity, 3, 9, O, Map(word -> rejects, sentence -> 0), List()), List(named_entity, 11, 16, B-MISC, Map(word -> German, sentence -> 0), List()), List(named_entity, 18, 21, O, Map(word -> call, sentence -> 0), List()), List(named_entity, 23, 24, O, Map(word -> to, sentence -> 0), List()), List(named_entity, 26, 32, O, Map(word -> boycott, sentence -> 0), List()), List(named_entity, 34, 40, B-MISC, Map(word -> British, sentence -> 0), List()), List(named_entity, 42, 45, O, Map(word -> lamb, sentence -> 0), List()), List(named_entity, 47, 47, O, Map(word -> ., sentence -> 0), List()))"
Peter Blackburn,"List(List(document, 0, 14, Peter Blackburn, Map(training -> true), List()))","List(List(document, 0, 14, Peter Blackburn, Map(sentence -> 0), List()))","List(List(token, 0, 4, Peter, Map(sentence -> 0), List()), List(token, 6, 14, Blackburn, Map(sentence -> 0), List()))","List(List(pos, 0, 4, NNP, Map(word -> Peter, sentence -> 0), List()), List(pos, 6, 14, NNP, Map(word -> Blackburn, sentence -> 0), List()))","List(List(named_entity, 0, 4, B-PER, Map(word -> Peter, sentence -> 0), List()), List(named_entity, 6, 14, I-PER, Map(word -> Blackburn, sentence -> 0), List()))"
BRUSSELS 1996-08-22,"List(List(document, 0, 18, BRUSSELS 1996-08-22, Map(training -> true), List()))","List(List(document, 0, 18, BRUSSELS 1996-08-22, Map(sentence -> 0), List()))","List(List(token, 0, 7, BRUSSELS, Map(sentence -> 0), List()), List(token, 9, 18, 1996-08-22, Map(sentence -> 0), List()))","List(List(pos, 0, 7, NNP, Map(word -> BRUSSELS, sentence -> 0), List()), List(pos, 9, 18, CD, Map(word -> 1996-08-22, sentence -> 0), List()))","List(List(named_entity, 0, 7, B-LOC, Map(word -> BRUSSELS, sentence -> 0), List()), List(named_entity, 9, 18, O, Map(word -> 1996-08-22, sentence -> 0), List()))"
The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .,"List(List(document, 0, 186, The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep ., Map(training -> true), List()))","List(List(document, 0, 186, The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep ., Map(sentence -> 0), List()))","List(List(token, 0, 2, 

In [0]:
%pip install mlflow

Python interpreter will be restarted.
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139103 sha256=fd99644004caad5ec0061f8badb4bd36a4230a72d6add8c0e10007c67ece843d
  Stored in directory: /root/.cache/pip/wheels/7b/ef/c5/85718fa9e66dec117e42d8b4d7b8a2e40ebdec17232935615f
Successfully built databricks-cli
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-15083cd9-4db1-4af6-97ff-45eb729d5bab
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Not uninstalling jinja2 at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-15083cd9-4db1-4af6-97ff-45eb729d5bab
    Can't uninstall 'Jinja2'. No files were found to unins

##Fit a pipeline on the training data

We are going to transform text into word embeddings, and then feed them into a named entity recognition transformer model.

### `WordEmbeddingsModel` ([link](https://nlp.johnsnowlabs.com/api/com/johnsnowlabs/nlp/embeddings/WordEmbeddingsModel))
> "Word Embeddings lookup annotator that maps tokens to vectors"

### `NerDlApproach` ([link](https://nlp.johnsnowlabs.com/api/com/johnsnowlabs/nlp/annotators/ner/dl/NerDLApproach))
> "This Named Entity recognition annotator allows to train generic NER model based on Neural Networks."

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

import mlflow
with mlflow.start_run() as mlflow_run:
 
  max_epochs=1
  lr=0.003
  batch_size=32
  random_seed=0
  verbose=1
  validation_split= 0.2
  evaluation_log_extended= True
  enable_output_logs= True
  include_confidence= True
  output_logs_path="dbfs:/ner_logs"

  dbutils.fs.mkdirs(output_logs_path)

  nerTagger = NerDLApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(max_epochs)\
    .setLr(lr)\
    .setBatchSize(batch_size)\
    .setRandomSeed(random_seed)\
    .setVerbose(verbose)\
    .setValidationSplit(validation_split)\
    .setEvaluationLogExtended(evaluation_log_extended)\
    .setEnableOutputLogs(enable_output_logs)\
    .setIncludeConfidence(include_confidence)\
    .setOutputLogsPath(output_logs_path)

  # Log model training parameters to MLflow.
  mlflow.log_params({
    "max_epochs": max_epochs,
    "lr": lr,
    "batch_size": batch_size,
    "random_seed": random_seed,
    "verbose": verbose,
    "validation_split": validation_split,
    "evaluation_log_extended": evaluation_log_extended,
    "enable_output_logs": enable_output_logs,
    "include_confidence": include_confidence,
    "output_logs_path": output_logs_path
  })

  # The training and evaluation data is already tokenized, so you can directly 
  # apply the embedding model and then fit a named-entity recognizer on the embeddings.
  glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
            .setInputCols(["document", "token"])\
            .setOutputCol("embeddings")

  ner_pipeline = Pipeline(stages=[
            glove_embeddings,
            nerTagger
   ])

  ner_model = ner_pipeline.fit(training_data)

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


## Evaluate on test data

In [0]:
predictions = ner_model.transform(test_data)

In [0]:
import pyspark.sql.functions as F
display(predictions.select(F.col('token.result').alias("tokens"),
                           F.col('label.result').alias("ground_truth"),
                           F.col('ner.result').alias("predictions")).limit(3))

tokens,ground_truth,predictions
"List(CRICKET, -, LEICESTERSHIRE, TAKE, OVER, AT, TOP, AFTER, INNINGS, VICTORY, .)","List(O, O, B-ORG, O, O, O, O, O, O, O, O)","List(O, O, B-ORG, O, O, O, O, O, O, O, O)"
"List(LONDON, 1996-08-30)","List(B-LOC, O)","List(B-LOC, O)"
"List(West, Indian, all-rounder, Phil, Simmons, took, four, for, 38, on, Friday, as, Leicestershire, beat, Somerset, by, an, innings, and, 39, runs, in, two, days, to, take, over, at, the, head, of, the, county, championship, .)","List(B-MISC, I-MISC, O, B-PER, I-PER, O, O, O, O, O, O, O, B-ORG, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O)","List(B-LOC, I-LOC, O, B-PER, I-PER, O, O, O, O, O, O, O, B-ORG, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O)"


In [0]:
# Reformat data to one token per row for evaluation.
predictions_pandas = predictions.select(F.explode(F.arrays_zip(predictions.token.result,
                                                     predictions.label.result,
                                                     predictions.ner.result)).alias("cols")) \
                              .select(F.expr("cols['0']").alias("token"),
                                      F.expr("cols['1']").alias("ground_truth"),
                                      F.expr("cols['2']").alias("prediction")).toPandas()

## Show the first twenty tokens and labels

In [0]:
display(predictions_pandas.head(20))

token,ground_truth,prediction
CRICKET,O,O
-,O,O
LEICESTERSHIRE,B-ORG,B-ORG
TAKE,O,O
OVER,O,O
AT,O,O
TOP,O,O
AFTER,O,O
INNINGS,O,O
VICTORY,O,O


## Construct and log a prediction pipeline for text

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
 
sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')
 
token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')
 
# Pull out the model from the pipeline.
loaded_ner_model = ner_model.stages[1]
 
converter = NerConverter()\
      .setInputCols(["document", "token", "ner"])\
      .setOutputCol("ner_span")
 
ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        glove_embeddings,
        loaded_ner_model,
        converter])

In [0]:
# Fitting with an empty data frame allows you to construct a pipeline model 
# without retraining the model.
empty_data = spark.createDataFrame([['']]).toDF("text")
prediction_model = ner_prediction_pipeline.fit(empty_data)

### Feed In A Test Document

In [0]:
# Create sample text.
text = "From the corner of the divan of Persian saddle-bags on which he was lying, smoking, as was his custom, innumerable cigarettes, Lord Henry Wotton could just catch the gleam of the honey-sweet and honey-coloured blossoms of a laburnum, whose tremulous branches seemed hardly able to bear the burden of a beauty so flamelike as theirs; and now and then the fantastic shadows of birds in flight flitted across the long tussore-silk curtains that were stretched in front of the huge window, producing a kind of momentary Japanese effect, and making him think of those pallid, jade-faced painters of Tokyo who, through the medium of an art that is necessarily immobile, seek to convey the sense of swiftness and motion. The sullen murmur of the bees shouldering their way through the long unmown grass, or circling with monotonous insistence round the dusty gilt horns of the straggling woodbine, seemed to make the stillness more oppressive. The dim roar of London was like the bourdon note of a distant organ."
sample_data = spark.createDataFrame([[text]]).toDF("text")
 
predictions = prediction_model.transform(sample_data)
display(predictions)

text document sentence token embeddings ner ner_span From the corner of the divan of Persian saddle-bags on which he was lying, smoking, as was his custom, innumerable cigarettes, Lord Henry Wotton could just catch the gleam of the honey-sweet and honey-coloured blossoms of a laburnum, whose tremulous branches seemed hardly able to bear the burden of a beauty so flamelike as theirs; and now and then the fantastic shadows of birds in flight flitted across the long tussore-silk curtains that were stretched in front of the huge window, producing a kind of momentary Japanese effect, and making him think of those pallid, jade-faced painters of Tokyo who, through the medium of an art that is necessarily immobile, seek to convey the sense of swiftness and motion. The sullen murmur of the bees shouldering their way through the long unmown grass, or circling with monotonous insistence round the dusty gilt horns of the straggling woodbine, seemed to make the stillness more oppressive. The dim roar of London was like the bourdon note of a distant organ. List(List(document, 0, 1004, From the corner of the divan of Persian saddle-bags on which he was lying, smoking, as was his custom, innumerable cigarettes, Lord Henry Wotton could just catch the gleam of the honey-sweet and honey-coloured blossoms of a laburnum, whose tremulous branches seemed hardly able to bear the burden of a beauty so flamelike as theirs; and now and then the fantastic shadows of birds in flight flitted across the long tussore-silk curtains that were stretched in front of the huge window, producing a kind of momentary Japanese effect, and making him think of those pallid, jade-faced painters of Tokyo who, through the medium of an art that is necessarily immobile, seek to convey the sense of swiftness and motion. The sullen murmur of the bees shouldering their way through the long unmown grass, or circling with monotonous insistence round the dusty gilt horns of the straggling woodbine, seemed to make the stillness more oppressive. The dim roar of London was like the bourdon note of a distant organ., Map(sentence -> 0), List())) List(List(document, 0, 331, From the corner of the divan of Persian saddle-bags on which he was lying, smoking, as was his custom, innumerable cigarettes, Lord Henry Wotton could just catch the gleam of the honey-sweet and honey-coloured blossoms of a laburnum, whose tremulous branches seemed hardly able to bear the burden of a beauty so flamelike as theirs;, Map(sentence -> 0), List()), List(document, 333, 712, and now and then the fantastic shadows of birds in flight flitted across the long tussore-silk curtains that were stretched in front of the huge window, producing a kind of momentary Japanese effect, and making him think of those pallid, jade-faced painters of Tokyo who, through the medium of an art that is necessarily immobile, seek to convey the sense of swiftness and motion., Map(sentence -> 1), List()), List(document, 714, 935, The sullen murmur of the bees shouldering their way through the long unmown grass, or circling with monotonous insistence round the dusty gilt horns of the straggling woodbine, seemed to make the stillness more oppressive., Map(sentence -> 2), List()), List(document, 937, 1004, The dim roar of London was like the bourdon note of a distant organ., Map(sentence -> 3), List())) List(List(token, 0, 3, From, Map(sentence -> 0), List()), List(token, 5, 7, the, Map(sentence -> 0), List()), List(token, 9, 14, corner, Map(sentence -> 0), List()), List(token, 16, 17, of, Map(sentence -> 0), List()), List(token, 19, 21, the, Map(sentence -> 0), List()), List(token, 23, 27, divan, Map(sentence -> 0), List()), List(token, 29, 30, of, Map(sentence -> 0), List()), List(token, 32, 38, Persian, Map(sentence -> 0), List()), List(token, 40, 50, saddle-bags, Map(sentence -> 0), List()), List(token, 52, 53, on, Map(sentence -> 0), List()), List(token, 55, 59, which, Map(sentence -> 0), List()), List(token, 61, 62, he, Map(sentenc

### Extracted Entities

In [0]:
display(predictions.select(F.explode(F.arrays_zip(predictions.ner_span.result,predictions.ner_span.metadata)).alias("entities")) 
      .select(F.expr("entities['0']").alias("chunk"),
              F.expr("entities['1'].entity").alias("entity")))

chunk,entity
Persian,MISC
Lord Henry Wotton,PER
Japanese,MISC
Tokyo,LOC
London,LOC
